In [87]:
from typing import Dict, TypedDict, List
import random
from langgraph.graph import StateGraph, START, END

In [88]:
class AgentState(TypedDict):
    player_name: str
    target: int
    guesses: List[int]
    attempts: 0
    lower_bound: int
    upper_bound: int

In [89]:
def set_target(state: AgentState) -> AgentState:
    state['target'] = random.randint(state["lower_bound"],state["upper_bound"])

    return state

def guess(state: AgentState) -> AgentState:
    state["guesses"].append(random.randint(state["lower_bound"],state["upper_bound"]))
    state["attempts"] += 1
    
    return state

def hint(state: AgentState) -> AgentState:
    if state["guesses"][-1] != state["target"] and state["attempts"] < 8:
        if state["guesses"][-1] > state["target"]:
            print("bigger")    
        else:
            print("smaller")
    else:
        if state["guesses"][-1] == state["target"]:
            print("correct!")
        else:
            print("exceed the limit")
    
    return state

def should_continue(state: AgentState) -> AgentState:
    if state["guesses"][-1] != state["target"] and state["attempts"] < 8:
        
        return "continue"
    else:
        
        return "end"
        
        
    

In [90]:
graph = StateGraph(AgentState)

graph.add_node("setup", set_target)
graph.add_node("guess", guess)
graph.add_node("hint_node", hint)

graph.add_edge("setup", "guess")
graph.add_edge("guess", "hint_node")
graph.add_conditional_edges(
    "hint_node",
    should_continue,
    {
        "continue": "guess",
        "end": END
    }
)

graph.set_entry_point("setup")

app = graph.compile()

In [91]:
result = app.invoke({"attempts": 0, "lower_bound":1, "upper_bound": 50, "player_name": "Sam", "guesses": []})
result

bigger
bigger
smaller
smaller
smaller
smaller
bigger
exceed the limit


{'player_name': 'Sam',
 'target': 20,
 'guesses': [32, 22, 10, 8, 6, 12, 37, 26],
 'attempts': 8,
 'lower_bound': 1,
 'upper_bound': 50}